In [2]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

def generate_keys():
    # 生成密钥对
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()

    # 存储私钥
    with open("private_key.pem", "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        ))

    # 存储公钥
    with open("server_public_key.pem", "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))

# 调用函数生成密钥
generate_keys()


In [ ]:
import socket
import ssl
import time
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend

def load_private_key():
    with open("private_key.pem", "rb") as key_file:
        return serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )

def create_order_info():
    return b"Order: Item1x1, Item2x2, Total: $100"

def create_payment_info():
    return b"Payment: CardNumber: xxxx-xxxx-xxxx-xxxx, Exp: 10/24"

def sign_data(private_key, data):
    return private_key.sign(
        data,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

def double_signature(private_key, order_info, payment_info):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(order_info)
    order_hash = digest.finalize()

    digest = hashes.Hash(hashes.SHA256())
    digest.update(payment_info)
    payment_hash = digest.finalize()

    combined_hash = order_hash + payment_hash
    return sign_data(private_key, combined_hash)

def encrypt_data(public_key, data):
    return public_key.encrypt(
        data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

def send_secure_message():
    private_key = load_private_key()
    order_info = create_order_info()
    payment_info = create_payment_info()

    with open("server_public_key.pem", "rb") as key_file:
        public_key = serialization.load_pem_public_key(
            key_file.read(),
            backend=default_backend()
        )

    signature = double_signature(private_key, order_info, payment_info)
    encrypted_order_info = encrypt_data(public_key, order_info)
    encrypted_payment_info = encrypt_data(public_key, payment_info)

    context = ssl.create_default_context()
    context.load_verify_locations('server.crt')

    with socket.create_connection(('localhost', 65432)) as sock:
        with context.wrap_socket(sock, server_hostname='localhost') as ssock:
            ssock.sendall(encrypted_order_info + encrypted_payment_info + signature)
            print("Data sent securely!")
            time.sleep(2)  # Ensure server has time to process data before closing
            # Optionally wait for a response
            response = ssock.recv(1024)
            print("Response from server:", response.decode())

        ssock.shutdown(socket.SHUT_RDWR)
        ssock.close()

if __name__ == "__main__":
    send_secure_message()


Data sent securely!
